In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.mit_bih import MITBIH
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss, ClassificationKNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [3]:
k = 8
tree_depth = 6
batch_size = 512
device = 'cuda'
train_data_path = r'<>/mitbih_train.csv'  # replace <> with the correct path of the dataset
test_data_path = r'<>/mitbih_test.csv'  # replace <> with the correct path of the dataset

In [4]:
train_data_iter = torch.utils.data.DataLoader(MITBIH(train_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True,
                                             drop_last=True)

test_data_iter = torch.utils.data.DataLoader(MITBIH(test_data_path),
                                             batch_size=batch_size,
                                             shuffle=True,
                                             num_workers=1,
                                             pin_memory=True)

In [5]:
class ConvBlock(nn.Module):
    def __init__(self):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
    def forward(self, x):
        y = x
        y = self.conv1(y)
        y = self.relu1(y)
        y = self.conv2(y)
        y = y + x
        y = self.relu2(y)
        y = self.pool(y)
        return y


class ECGModel(nn.Module):
    def __init__(self):
        super(ECGModel, self).__init__()
        self.conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=1)
        self.block1 = ConvBlock()
        self.block2 = ConvBlock()
        self.block3 = ConvBlock()
        self.block4 = ConvBlock()
        self.block5 = ConvBlock()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 5)

    def forward(self, x, return_interm=False):
        x = self.conv(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        interm = x.flatten(1)
        x = self.fc1(interm)
        x = self.relu(x)
        x = self.fc2(x)
        
        if return_interm:
            return x, interm
        
        return x

In [6]:
knn_crt = ClassificationKNNLoss(k=k).to(device)

def train(model, loader, optimizer, device):
    model.train()

    total_loss = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        outputs, interm = model(batch, return_interm=True)
        mse_loss = F.cross_entropy(outputs, target)
        mse_loss = mse_loss.sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(interm, target)
            if torch.isinf(knn_loss):
                knn_loss = torch.tensor(0).to(device)
        except ValueError:
            knn_loss = torch.tensor(0).to(device)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(loader)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | CLS Loss: {mse_loss.item()}")

    return total_loss / len(loader)

@torch.no_grad()
def test(model, loader, device):
    model.eval()
    
    correct = 0
    for iteration, (batch, target) in enumerate(loader):
        batch = batch.to(device)
        target = target.to(device)
        y_pred = model(batch).argmax(dim=-1)
        correct += y_pred.eq(target.view(-1).data).sum()
    
    return correct / len(loader.dataset)

In [9]:
epochs = 200
lr = 1e-3
log_every = 10

model = ECGModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
num_params = sum([p.numel() for p in model.parameters()])
print(f'#Params: {num_params}')

#Params: 53957


In [11]:
best_valid_acc = 0
losses = []
train_accs = []
val_accs = []
for epoch in range(1, epochs + 1):
    loss = train(model, train_data_iter, optimizer, device)
#     print(f"Loss: {loss} =============================")
    losses.append(loss)
    train_acc = test(model, train_data_iter, device)
    train_accs.append(train_acc)
    valid_acc = test(model, test_data_iter, device)
    val_accs.append(valid_acc)
    if valid_acc > best_valid_acc:
        best_valid_acc = valid_acc
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
              f'Train: {train_acc:.4f}, Valid: {valid_acc:.4f}, '
              f'Best: {best_valid_acc:.4f}')

Epoch 1 / 200 | iteration 0 / 171 | Total Loss: 7.52977180480957 | KNN Loss: 5.478849411010742 | CLS Loss: 2.050922155380249
Epoch 1 / 200 | iteration 10 / 171 | Total Loss: 3.683405876159668 | KNN Loss: 2.786168098449707 | CLS Loss: 0.8972376585006714
Epoch 1 / 200 | iteration 20 / 171 | Total Loss: 3.446229934692383 | KNN Loss: 2.6365323066711426 | CLS Loss: 0.8096975684165955
Epoch 1 / 200 | iteration 30 / 171 | Total Loss: 3.278867721557617 | KNN Loss: 2.509679079055786 | CLS Loss: 0.769188642501831
Epoch 1 / 200 | iteration 40 / 171 | Total Loss: 3.1989879608154297 | KNN Loss: 2.5724802017211914 | CLS Loss: 0.6265076994895935
Epoch 1 / 200 | iteration 50 / 171 | Total Loss: 3.1702568531036377 | KNN Loss: 2.635077953338623 | CLS Loss: 0.5351788997650146
Epoch 1 / 200 | iteration 60 / 171 | Total Loss: 3.150221586227417 | KNN Loss: 2.5995898246765137 | CLS Loss: 0.5506318211555481
Epoch 1 / 200 | iteration 70 / 171 | Total Loss: 3.1188483238220215 | KNN Loss: 2.5542352199554443 | CL

Epoch 4 / 200 | iteration 80 / 171 | Total Loss: 2.6416022777557373 | KNN Loss: 2.507258892059326 | CLS Loss: 0.1343434751033783
Epoch 4 / 200 | iteration 90 / 171 | Total Loss: 2.6252903938293457 | KNN Loss: 2.466026544570923 | CLS Loss: 0.15926380455493927
Epoch 4 / 200 | iteration 100 / 171 | Total Loss: 2.632112503051758 | KNN Loss: 2.4795005321502686 | CLS Loss: 0.15261198580265045
Epoch 4 / 200 | iteration 110 / 171 | Total Loss: 2.6179697513580322 | KNN Loss: 2.491077184677124 | CLS Loss: 0.12689253687858582
Epoch 4 / 200 | iteration 120 / 171 | Total Loss: 2.622281074523926 | KNN Loss: 2.503100633621216 | CLS Loss: 0.11918036639690399
Epoch 4 / 200 | iteration 130 / 171 | Total Loss: 2.625605583190918 | KNN Loss: 2.4916012287139893 | CLS Loss: 0.13400425016880035
Epoch 4 / 200 | iteration 140 / 171 | Total Loss: 2.697336435317993 | KNN Loss: 2.5071699619293213 | CLS Loss: 0.1901664286851883
Epoch 4 / 200 | iteration 150 / 171 | Total Loss: 2.6291849613189697 | KNN Loss: 2.52084

Epoch 7 / 200 | iteration 160 / 171 | Total Loss: 2.5521764755249023 | KNN Loss: 2.437957763671875 | CLS Loss: 0.11421866714954376
Epoch 7 / 200 | iteration 170 / 171 | Total Loss: 2.560342788696289 | KNN Loss: 2.450408935546875 | CLS Loss: 0.10993392020463943
Epoch: 007, Loss: 2.5501, Train: 0.9765, Valid: 0.9739, Best: 0.9739
Epoch 8 / 200 | iteration 0 / 171 | Total Loss: 2.5339887142181396 | KNN Loss: 2.462736129760742 | CLS Loss: 0.07125262171030045
Epoch 8 / 200 | iteration 10 / 171 | Total Loss: 2.5076987743377686 | KNN Loss: 2.4443304538726807 | CLS Loss: 0.06336834281682968
Epoch 8 / 200 | iteration 20 / 171 | Total Loss: 2.5391862392425537 | KNN Loss: 2.462339162826538 | CLS Loss: 0.07684702426195145
Epoch 8 / 200 | iteration 30 / 171 | Total Loss: 2.537283420562744 | KNN Loss: 2.4383511543273926 | CLS Loss: 0.0989321619272232
Epoch 8 / 200 | iteration 40 / 171 | Total Loss: 2.5249199867248535 | KNN Loss: 2.42889666557312 | CLS Loss: 0.096023328602314
Epoch 8 / 200 | iteratio

Epoch 11 / 200 | iteration 50 / 171 | Total Loss: 2.533390760421753 | KNN Loss: 2.4464941024780273 | CLS Loss: 0.08689658343791962
Epoch 11 / 200 | iteration 60 / 171 | Total Loss: 2.504801034927368 | KNN Loss: 2.42911434173584 | CLS Loss: 0.07568667083978653
Epoch 11 / 200 | iteration 70 / 171 | Total Loss: 2.5146284103393555 | KNN Loss: 2.4148519039154053 | CLS Loss: 0.09977652877569199
Epoch 11 / 200 | iteration 80 / 171 | Total Loss: 2.5232040882110596 | KNN Loss: 2.4557511806488037 | CLS Loss: 0.06745290011167526
Epoch 11 / 200 | iteration 90 / 171 | Total Loss: 2.4928107261657715 | KNN Loss: 2.41213321685791 | CLS Loss: 0.08067762106657028
Epoch 11 / 200 | iteration 100 / 171 | Total Loss: 2.5432424545288086 | KNN Loss: 2.459280252456665 | CLS Loss: 0.08396217226982117
Epoch 11 / 200 | iteration 110 / 171 | Total Loss: 2.536130666732788 | KNN Loss: 2.4660532474517822 | CLS Loss: 0.07007752358913422
Epoch 11 / 200 | iteration 120 / 171 | Total Loss: 2.5211100578308105 | KNN Loss: 

Epoch 14 / 200 | iteration 120 / 171 | Total Loss: 2.4870569705963135 | KNN Loss: 2.4060440063476562 | CLS Loss: 0.0810130164027214
Epoch 14 / 200 | iteration 130 / 171 | Total Loss: 2.472944974899292 | KNN Loss: 2.4321982860565186 | CLS Loss: 0.04074675589799881
Epoch 14 / 200 | iteration 140 / 171 | Total Loss: 2.482830047607422 | KNN Loss: 2.402920961380005 | CLS Loss: 0.07990897446870804
Epoch 14 / 200 | iteration 150 / 171 | Total Loss: 2.436797618865967 | KNN Loss: 2.4038889408111572 | CLS Loss: 0.032908715307712555
Epoch 14 / 200 | iteration 160 / 171 | Total Loss: 2.4863297939300537 | KNN Loss: 2.436267375946045 | CLS Loss: 0.050062377005815506
Epoch 14 / 200 | iteration 170 / 171 | Total Loss: 2.4573404788970947 | KNN Loss: 2.401641607284546 | CLS Loss: 0.05569878965616226
Epoch: 014, Loss: 2.4987, Train: 0.9845, Valid: 0.9803, Best: 0.9803
Epoch 15 / 200 | iteration 0 / 171 | Total Loss: 2.478961706161499 | KNN Loss: 2.4204139709472656 | CLS Loss: 0.05854771286249161
Epoch 15

...

Epoch 186 / 200 | iteration 10 / 171 | Total Loss: 2.423741102218628 | KNN Loss: 2.4106104373931885 | CLS Loss: 0.01313056517392397
Epoch 186 / 200 | iteration 20 / 171 | Total Loss: 2.4519309997558594 | KNN Loss: 2.440330982208252 | CLS Loss: 0.011600104160606861
Epoch 186 / 200 | iteration 30 / 171 | Total Loss: 2.422361373901367 | KNN Loss: 2.4131765365600586 | CLS Loss: 0.009184829890727997
Epoch 186 / 200 | iteration 40 / 171 | Total Loss: 2.4058241844177246 | KNN Loss: 2.3885395526885986 | CLS Loss: 0.017284715548157692
Epoch 186 / 200 | iteration 50 / 171 | Total Loss: 2.423069953918457 | KNN Loss: 2.4066174030303955 | CLS Loss: 0.016452541574835777
Epoch 186 / 200 | iteration 60 / 171 | Total Loss: 2.419846534729004 | KNN Loss: 2.39336895942688 | CLS Loss: 0.02647750824689865
Epoch 186 / 200 | iteration 70 / 171 | Total Loss: 2.4381065368652344 | KNN Loss: 2.4240856170654297 | CLS Loss: 0.014020973816514015
Epoch 186 / 200 | iteration 80 / 171 | Total Loss: 2.3900530338287354 |

Epoch 189 / 200 | iteration 70 / 171 | Total Loss: 2.4003379344940186 | KNN Loss: 2.3877363204956055 | CLS Loss: 0.012601522728800774
Epoch 189 / 200 | iteration 80 / 171 | Total Loss: 2.4133224487304688 | KNN Loss: 2.3927459716796875 | CLS Loss: 0.020576534792780876
Epoch 189 / 200 | iteration 90 / 171 | Total Loss: 2.3987808227539062 | KNN Loss: 2.3797974586486816 | CLS Loss: 0.01898333989083767
Epoch 189 / 200 | iteration 100 / 171 | Total Loss: 2.445039987564087 | KNN Loss: 2.4145350456237793 | CLS Loss: 0.03050500713288784
Epoch 189 / 200 | iteration 110 / 171 | Total Loss: 2.433316230773926 | KNN Loss: 2.419560670852661 | CLS Loss: 0.013755575753748417
Epoch 189 / 200 | iteration 120 / 171 | Total Loss: 2.501502275466919 | KNN Loss: 2.4793009757995605 | CLS Loss: 0.02220122143626213
Epoch 189 / 200 | iteration 130 / 171 | Total Loss: 2.442021608352661 | KNN Loss: 2.430443525314331 | CLS Loss: 0.011578013189136982
Epoch 189 / 200 | iteration 140 / 171 | Total Loss: 2.4157562255859

Epoch 192 / 200 | iteration 130 / 171 | Total Loss: 2.4154891967773438 | KNN Loss: 2.40907883644104 | CLS Loss: 0.0064102946780622005
Epoch 192 / 200 | iteration 140 / 171 | Total Loss: 2.41257643699646 | KNN Loss: 2.407850503921509 | CLS Loss: 0.004725826904177666
Epoch 192 / 200 | iteration 150 / 171 | Total Loss: 2.377790689468384 | KNN Loss: 2.354823589324951 | CLS Loss: 0.02296704612672329
Epoch 192 / 200 | iteration 160 / 171 | Total Loss: 2.392366647720337 | KNN Loss: 2.384329080581665 | CLS Loss: 0.008037499152123928
Epoch 192 / 200 | iteration 170 / 171 | Total Loss: 2.438843250274658 | KNN Loss: 2.4248478412628174 | CLS Loss: 0.01399534847587347
Epoch: 192, Loss: 2.4226, Train: 0.9970, Valid: 0.9871, Best: 0.9877
Epoch 193 / 200 | iteration 0 / 171 | Total Loss: 2.4052398204803467 | KNN Loss: 2.3996529579162598 | CLS Loss: 0.005586802028119564
Epoch 193 / 200 | iteration 10 / 171 | Total Loss: 2.400081157684326 | KNN Loss: 2.3848702907562256 | CLS Loss: 0.015210882760584354
E

Epoch 196 / 200 | iteration 10 / 171 | Total Loss: 2.3891751766204834 | KNN Loss: 2.378584623336792 | CLS Loss: 0.010590544901788235
Epoch 196 / 200 | iteration 20 / 171 | Total Loss: 2.4515790939331055 | KNN Loss: 2.446146011352539 | CLS Loss: 0.005433198995888233
Epoch 196 / 200 | iteration 30 / 171 | Total Loss: 2.3746628761291504 | KNN Loss: 2.368917942047119 | CLS Loss: 0.005744998808950186
Epoch 196 / 200 | iteration 40 / 171 | Total Loss: 2.405022621154785 | KNN Loss: 2.3930423259735107 | CLS Loss: 0.011980370618402958
Epoch 196 / 200 | iteration 50 / 171 | Total Loss: 2.385594129562378 | KNN Loss: 2.36598801612854 | CLS Loss: 0.01960601843893528
Epoch 196 / 200 | iteration 60 / 171 | Total Loss: 2.4038076400756836 | KNN Loss: 2.398301362991333 | CLS Loss: 0.005506326910108328
Epoch 196 / 200 | iteration 70 / 171 | Total Loss: 2.469566822052002 | KNN Loss: 2.4619991779327393 | CLS Loss: 0.00756765715777874
Epoch 196 / 200 | iteration 80 / 171 | Total Loss: 2.375807523727417 | KN

Epoch 199 / 200 | iteration 70 / 171 | Total Loss: 2.4101948738098145 | KNN Loss: 2.392791748046875 | CLS Loss: 0.017403146252036095
Epoch 199 / 200 | iteration 80 / 171 | Total Loss: 2.3714137077331543 | KNN Loss: 2.3656351566314697 | CLS Loss: 0.0057786256074905396
Epoch 199 / 200 | iteration 90 / 171 | Total Loss: 2.4124298095703125 | KNN Loss: 2.402292251586914 | CLS Loss: 0.010137462988495827
Epoch 199 / 200 | iteration 100 / 171 | Total Loss: 2.4406585693359375 | KNN Loss: 2.4180727005004883 | CLS Loss: 0.022585898637771606
Epoch 199 / 200 | iteration 110 / 171 | Total Loss: 2.4331226348876953 | KNN Loss: 2.419377326965332 | CLS Loss: 0.013745265081524849
Epoch 199 / 200 | iteration 120 / 171 | Total Loss: 2.4068851470947266 | KNN Loss: 2.3972902297973633 | CLS Loss: 0.009594847448170185
Epoch 199 / 200 | iteration 130 / 171 | Total Loss: 2.4206607341766357 | KNN Loss: 2.404902935028076 | CLS Loss: 0.015757769346237183
Epoch 199 / 200 | iteration 140 / 171 | Total Loss: 2.4278264

In [12]:
test(model, test_data_iter, device)

tensor(0.9856, device='cuda:0')

In [13]:
plt.figure()
plt.plot(losses, label='train loss')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plt.figure()
plt.plot(train_accs, label='train accuracy')
plt.plot(val_accs, label='validation accuracy')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
test_samples = torch.tensor([])
projections = torch.tensor([])
labels = torch.tensor([])

with torch.no_grad():
    for x, y in tqdm(test_data_iter):
        test_samples = torch.cat([test_samples, x])
        labels = torch.cat([labels, y])
        x = x.to(device)
        _, interm = model(x, True)
        projections = torch.cat([projections, interm.detach().cpu().flatten(1)])

  0%|          | 0/43 [00:00<?, ?it/s]

In [16]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
clusters = DBSCAN(eps=2, min_samples=10).fit_predict(projections)

In [18]:
print(f"Number of inliers: {sum(clusters != -1) / len(clusters)}")

Number of inliers: 0.9099629984925311


In [19]:
perplexity = 100
p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [20]:
tree_dataset = list(zip(test_samples.flatten(1)[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [21]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [22]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [23]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 400
log_interval = 100
use_cuda = device != 'cpu'

In [24]:
tree = SDT(input_dim=test_samples.shape[2], output_dim=len(set(clusters)) - 1, depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [25]:
losses = []
accs = []
sparsity = []

In [26]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 039 | Total loss: 3.270 | Reg loss: 0.007 | Tree loss: 3.270 | Accuracy: 0.046875 | 0.091 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 01 | Batch: 000 / 039 | Total loss: 3.193 | Reg loss: 0.005 | Tree loss: 3.193 | Accuracy: 0.136719 | 0.063 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 02 | Batch: 000 / 039 | Total loss: 3.135 | Reg loss: 0.008 | Tree loss: 3.135 | Accuracy: 0.146484 | 0.062 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 28 | Batch: 000 / 039 | Total loss: 2.743 | Reg loss: 0.023 | Tree loss: 2.743 | Accuracy: 0.216797 | 0.062 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 29 | Batch: 000 / 039 | Total loss: 2.686 | Reg loss: 0.023 | Tree loss: 2.686 | Accuracy: 0.267578 | 0.062 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 30 | Batch: 000 / 039 | Total loss: 2.647 | Reg loss: 0.023 | Tree loss: 2.647 | Accuracy: 0.304688 | 0.062 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 56 | Batch: 000 / 039 | Total loss: 2.454 | Reg loss: 0.024 | Tree loss: 2.454 | Accuracy: 0.283203 | 0.058 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 57 | Batch: 000 / 039 | Total loss: 2.436 | Reg loss: 0.024 | Tree loss: 2.436 | Accuracy: 0.296875 | 0.057 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 58 | Batch: 000 / 039 | Total loss: 2.474 | Reg loss: 0.024 | Tree loss: 2.474 | Accuracy: 0.285156 | 0.057 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 84 | Batch: 000 / 039 | Total loss: 2.435 | Reg loss: 0.025 | Tree loss: 2.435 | Accuracy: 0.294922 | 0.054 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 85 | Batch: 000 / 039 | Total loss: 2.439 | Reg loss: 0.025 | Tree loss: 2.439 | Accuracy: 0.300781 | 0.054 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 86 | Batch: 000 / 039 | Total loss: 2.442 | Reg loss: 0.025 | Tree loss: 2.442 | Accuracy: 0.302734 | 0.054 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 112 | Batch: 000 / 039 | Total loss: 2.385 | Reg loss: 0.025 | Tree loss: 2.385 | Accuracy: 0.294922 | 0.052 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 113 | Batch: 000 / 039 | Total loss: 2.428 | Reg loss: 0.025 | Tree loss: 2.428 | Accuracy: 0.300781 | 0.052 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 114 | Batch: 000 / 039 | Total loss: 2.449 | Reg loss: 0.025 | Tree loss: 2.449 | Accuracy: 0.273438 | 0.052 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

...

Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 280 | Batch: 000 / 039 | Total loss: 2.139 | Reg loss: 0.028 | Tree loss: 2.139 | Accuracy: 0.396484 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 281 | Batch: 000 / 039 | Total loss: 2.096 | Reg loss: 0.028 | Tree loss: 2.096 | Accuracy: 0.400391 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 282 | Batch: 000 / 039 | Total loss: 2.188 | Reg loss: 0.028 | Tree loss: 2.188 | Accuracy: 0.373047 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 308 | Batch: 000 / 039 | Total loss: 2.106 | Reg loss: 0.027 | Tree loss: 2.106 | Accuracy: 0.400391 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 309 | Batch: 000 / 039 | Total loss: 2.086 | Reg loss: 0.027 | Tree loss: 2.086 | Accuracy: 0.431641 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 310 | Batch: 000 / 039 | Total loss: 2.049 | Reg loss: 0.027 | Tree loss: 2.049 | Accuracy: 0.417969 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 336 | Batch: 000 / 039 | Total loss: 2.083 | Reg loss: 0.027 | Tree loss: 2.083 | Accuracy: 0.423828 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 337 | Batch: 000 / 039 | Total loss: 2.132 | Reg loss: 0.027 | Tree loss: 2.132 | Accuracy: 0.414062 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 338 | Batch: 000 / 039 | Total loss: 2.030 | Reg loss: 0.027 | Tree loss: 2.030 | Accuracy: 0.472656 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 364 | Batch: 000 / 039 | Total loss: 2.151 | Reg loss: 0.027 | Tree loss: 2.151 | Accuracy: 0.394531 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 365 | Batch: 000 / 039 | Total loss: 2.134 | Reg loss: 0.027 | Tree loss: 2.134 | Accuracy: 0.392578 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 366 | Batch: 000 / 039 | Total loss: 2.060 | Reg loss: 0.027 | Tree loss: 2.060 | Accuracy: 0.431641 | 0.06 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894


Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 392 | Batch: 000 / 039 | Total loss: 2.094 | Reg loss: 0.027 | Tree loss: 2.094 | Accuracy: 0.423828 | 0.059 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 393 | Batch: 000 / 039 | Total loss: 2.139 | Reg loss: 0.027 | Tree loss: 2.139 | Accuracy: 0.408203 | 0.059 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.9840425531914894
layer 2: 0.9840425531914894
layer 3: 0.9840425531914894
layer 4: 0.9840425531914894
Epoch: 394 | Batch: 000 / 039 | Total loss: 2.114 | Reg loss: 0.027 | Tree loss: 2.114 | Accuracy: 0.417969 | 0.059 sec/iter
Average sparseness: 0.984042553191489
layer 0: 0.9840425531914894
layer 1: 0.98404255319148

In [27]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
# plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [29]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.222222222222222


# Extract Rules

# Accumulate samples in the leaves

In [30]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 18


In [31]:
method = 'greedy'

In [32]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [35]:
attr_names = [f"T_{i}" for i in range(test_samples.shape[2])]
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
254
============== Pattern 2 ==============
335
============== Pattern 3 ==============
1302
============== Pattern 4 ==============
1420
============== Pattern 5 ==============
============== Pattern 6 ==============
626
============== Pattern 7 ==============
2427
============== Pattern 8 ==============
============== Pattern 9 ==============
5248
============== Pattern 10 ==============
5396
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
526
============== Pattern 16 ==============
2386
============== Pattern 17 ==============
============== Pattern 18 ==============
Average comprehensibility: 28.333333333333332
std comprehensibility: 8.034647195462632
var comprehensibility: 64.55555555555556
minimum comprehensibility: 4
maximum comprehensibility: 34
